# Anime GAN

## Load Python Dependencies

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import sys
from imageio import imread
import time
import scipy.misc
import os

## Load Tensorflow/Keras

In [2]:
from tensorflow.keras.models import load_model
import tensorflow as tf

## Set System Settings

In [3]:
from tensorflow import set_random_seed
np.random.seed(69)
set_random_seed(69)
%matplotlib inline

## Set Constants

In [4]:
LATENT_DIM = 32              # Dimension of the latent space from which to sample
IMAGE_DIM = 64                # Height and width of images

## Paths

In [5]:
MODEL_NAME = "gan.anime.{}.{}.h5"                    # Model filename
DATA_DIR = '../0_data/anime_faces/'                  # Location of the training image files
IMAGE_FILENAMES_FILE = '../0_data/names.txt'         # List of image file names
PIPELINE_DIR = '../2_pipeline/1_anime_GAN_sampling/' # Where output are to be saved
OUT_DIR = PIPELINE_DIR + 'out/'                      # output director
STORE_DIR = PIPELINE_DIR + 'store/'                  # store directory
TMP_DIR = PIPELINE_DIR + 'tmp/'                      # temp directory
G_MODEL_PATH = '../2_pipeline/0_anime_GAN/out/CHECKPOINT_ANIME_GAN.G.gen00500.h5'

## Helper Functions

In [6]:
def make_dir_safely(dirname):
    """
    Given a directory name, create that directory if it does not exist
    """
    if not os.path.exists(dirname):
        os.makedirs(dirname)

def plot_images(ims, filename, random=False, lim=6):
    """
    A function for plotting a series of images
    """
    im_dim = ims.shape[1]
    if len(ims) < lim**2:
        indices = np.arange(len(ims))
    elif random:
        indices = np.arange(len(ims))
        np.random.shuffle(indices)
        indices = indices[:lim**2]
        indices = np.random.randint(0, len(ims), lim**2)
    else:
        indices = np.arange(lim**2)

    full_im = np.zeros((lim*im_dim, lim*im_dim, 3))
    for index, i in enumerate(indices):
        c, r = index % lim, index // lim
        full_im[r*im_dim:(r+1)*im_dim, c*im_dim:(c+1)*im_dim] = ims[i, :, :, :]
            
    scipy.misc.toimage(full_im, cmin=0.0, cmax=1.0).save(filename)

## Random Vector Function

In [7]:
def get_noise(num=32, dim=LATENT_DIM):
    """ Return a shape (num, dum) noise vector N(0, I)"""
    return np.random.normal(0, 1, (num, dim))

## Set Up Directories For Output

In [8]:
make_dir_safely(PIPELINE_DIR)
make_dir_safely(TMP_DIR)
make_dir_safely(STORE_DIR)

make_dir_safely(OUT_DIR)
make_dir_safely(OUT_DIR + 'raw_samples/')
make_dir_safely(OUT_DIR + 'interpolation/')

## Load Model

In [9]:
G = load_model(G_MODEL_PATH)

Instructions for updating:
Colocations handled automatically by placer.


## Plot images

In [10]:
for i in range(10):
    noise = get_noise(36, LATENT_DIM)
    ims = G.predict(noise)
    plot_images(ims, '{}raw_samples/sample.{:02d}.png'.format(OUT_DIR, i+1), lim=6)

D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


In [11]:
import shutil
temp_interpolate_path = TMP_DIR + 'interpolate/'
scale = 30
num = 10

for i in range(10):
    if os.path.exists(temp_interpolate_path):
        shutil.rmtree(temp_interpolate_path)
    make_dir_safely(temp_interpolate_path)
    vecs = get_noise(num, LATENT_DIM)
    vecs = np.concatenate([vecs, vecs[0:1]])
    
    interps = np.zeros((num*scale+1, LATENT_DIM))
    for j in range(num):
        interps[(j)*scale:(j+1)*scale+1:] = np.linspace(vecs[j], vecs[j+1], scale+1)
    interps = interps[:-1]
    #print(interps[-0], '\n', interps[-2], '\n', interps[-1])
    ims = G.predict(interps)
    for j in range(ims.shape[0]):
        plot_images(ims[j:j+1], '{}im.{:03d}.png'.format(temp_interpolate_path, j), lim=1)
    command = ('ffmpeg -i ' + 
              temp_interpolate_path + 'im.%03d.png -r 30 ' 
              + OUT_DIR + 'interpolation/sample_' + str(i) + '_interp.gif')
    print(os.system(command))

shutil.rmtree(temp_interpolate_path)

    

D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


0
0
0
0
0
0
0
0
0
0
